In [ ]:
import numpy as np
f = open("train/labels.txt")
content = f.readlines()
classes = []
for i in content:
    for j in i.replace('\n','').split(','):
        classes.append(j)
classes = np.unique(np.array(classes))
classes.shape

In [194]:
from scipy import ndimage
import cv2
import numpy as np
from math import atan2, cos, sin, sqrt, pi
import math

def GetIndices(img):
    
    bg=img[0,0]
    
    set1=np.array([],int)
    for i in range(200):
        if np.sum(np.array(img[:,i]!=bg))>50:
            set1=np.append(set1,i)       
    pos1=int(np.mean(set1))


    set2=np.array([],int)
    for i in range(pos1+50,350):
        if np.sum(np.array(img[:,i]!=bg))>50:
            set2=np.append(set2,i)       
    pos2=int(np.mean(set2))



    set3=np.array([],int)
    for i in range(pos2+50,500):
        if np.sum(np.array(img[:,i]!=bg))>50:
            set3=np.append(set3,i)
    pos3=int(np.mean(set3))
    

    ## Below are the row and colum slicing indices for the three segments img[row:row, column:column]
#     segment1=[5:175,pos1-70:pos1+70]
#     segment2=[5:175,pos2-70:pos2+70]
#     segment3=[5:175,pos3-70:pos3+70]
    
    segment1=[pos1-70,pos1+70]
    segment2=[pos2-70,pos2+70]
    segment3=[pos3-70,pos3+70]
    
    return [segment1,segment2,segment3]

def getOrientation(pts, img):
    sz = len(pts)
    data_pts = np.empty((sz, 2), dtype=np.float64)
    for i in range(data_pts.shape[0]):
        data_pts[i,0] = pts[i,0,0]
        data_pts[i,1] = pts[i,0,1]
    mean = np.empty((0))
    mean, eigenvectors, eigenvalues = cv2.PCACompute2(data_pts, mean)
    cntr = (int(mean[0,0]), int(mean[0,1]))
    angle = atan2(eigenvectors[0,1], eigenvectors[0,0])
    return angle



def rotation(ximg):
    #img = ximg.astype(np.uint8)
    #_, bw = cv2.threshold(gray, 50, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    img = ximg.astype(np.uint8)
    contours, _ = cv2.findContours(img, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
    
    
    max_angle = 0
    for i, c in enumerate(contours):
        area = cv2.contourArea(c)
        if area < 40 or 1e4 < area:
            continue

        angle = getOrientation(c, img)
        angle = math.ceil(angle*180/math.pi)
        rect = cv2.minAreaRect(c)
        box = cv2.boxPoints(rect)
        box = np.int0(box)


        center = (int(rect[0][0]),int(rect[0][1]))
        width = int(rect[1][0])
        height = int(rect[1][1])
        #angle = int(rect[2])
        
        if(angle>45):
            angle = -90+angle
        if(angle<-45):
            angle = -angle
        if(abs(angle) > abs(max_angle)):
            if(abs(angle)<= 32):
                max_angle = angle
            
    
    M = cv2.getRotationMatrix2D((img.shape[1]//2, img.shape[0]//2), max_angle, 1)
    img = cv2.warpAffine(ximg, M, (img.shape[1], img.shape[0]), cv2.INTER_LINEAR)
    return img, max_angle

def get_center_of_mass(img):
        Y,X = ndimage.measurements.center_of_mass(img)
        x,y = img.shape
        delta_x = np.round(y/2.0-X).astype(int)
        delta_y = np.round(x/2.0-Y).astype(int)
        return delta_x, delta_y
    
def get_to_center(image ,x, y):

        (rows , cols) = image.shape
        M = np.float32([[1,0,x],[0,1,y]])
        centered = cv2.warpAffine(image,M,(cols,rows))
        return centered 

def initial_preprocess(img):   # give raw images
    image = cv2.resize(img, (150,150))
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    (T, threshInv) = cv2.threshold(gray.copy(), 127, 255,
        cv2.THRESH_BINARY_INV )

    kernel_2 = cv2.getStructuringElement(cv2.MORPH_RECT,(3,3))
    cimg = cv2.erode(threshInv, kernel_2, iterations= 1)
    return cimg

def translation(cimg):  # give thresholded images from 

        while np.sum(cimg[0]) == 0:  #making squared image with respective pixels
            cimg = cimg[1:]

        while np.sum(cimg[0,:]) == 0:
            cimg = cimg[:,1:]

        while np.sum(cimg[-1]) == 0:
            cimg = cimg[:-1]

        while np.sum(cimg[:, -1])==0:
            cimg = cimg[:,:-1]
            
        rows,cols = cimg.shape
      
        if rows == cols:
            nrows = 130
            ncols = 130
            cimg = cv2.resize(cimg, (ncols,nrows))
           

        if rows > cols:
            nrows = 130
            ncols = int(round((cols*130.0/rows), 0))
            cimg = cv2.resize(cimg, (ncols,nrows))
            
        else:
            ncols = 130
            nrows = int(round((rows*130.0/cols), 0))
            
            cimg = cv2.resize(cimg, (ncols,nrows))
            
                             
        
        col_pad = (int(math.ceil((150-ncols)/2.0)), int(math.floor((150-ncols)/2.0)))

        row_pad = (int(math.ceil((150-nrows)/2.0)), int(math.floor((150-nrows)/2.0)))
        cimg = np.lib.pad(cimg,(row_pad,col_pad),'constant')


        del_x, del_y = get_center_of_mass(cimg) 
        centered = get_to_center(cimg ,del_x, del_y)
         
        ximg = centered.reshape(1,150,150).astype(np.float32)
        ximg-= int(33.3952)
        ximg/= int(78.6662)
        return ximg[0]
#img, angle = rotate_image(X)

In [219]:
import os
import numpy as np
import cv2
import math
from scipy import ndimage
from keras_preprocessing.image import load_img, img_to_array, array_to_img

s1 = 150
s2 = 150
classes = os.listdir("t5")
y = []
x = np.zeros(shape=(6000, s1, s1),dtype=np.float32)
k = 0
for i in classes:
    imgs = os.listdir(f"t5/{i}")
    for j in imgs:
        img = cv2.imread(f"t5/{i}/{j}")
        iimg = initial_preprocess(img)
        timg = translation(iimg)
        fimg,_ = rotation(timg)
        #img, _ = preprocessing_test(img)
        x[k] = fimg #255-threshInv #img_to_array(img)
        y.append(i) 
        k += 1

C:\Users\ambat\AppData\Local\Temp\ipykernel_89852\278173202.py:97: DeprecationWarning: Please use `center_of_mass` from the `scipy.ndimage` namespace, the `scipy.ndimage.measurements` namespace is deprecated.
  Y,X = ndimage.measurements.center_of_mass(img)


In [210]:
k = 28
print(y[k])
print(x[k].shape)
cv2.imshow('train',x[k])
cv2.waitKey()

ALPHA
(150, 150)


-1

In [221]:
y = np.array(y)

In [222]:
print(x.shape)
print(y.shape)

(6000, 150, 150)
(6000,)


In [223]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
print(X_train.shape)
print(X_test.shape)

(4800, 150, 150)
(1200, 150, 150)


In [16]:
#X_test_pre = np.zeros(shape=(1200, 150, 150))
#print(X_test_pre.shape)
#for idx, i in enumerate(X_test):
#    
#    im,_ = rotation(i)
#    X_test_pre[idx] = im
#X_test_pre.shape

(1200, 150, 150)


(1200, 150, 150)

In [7]:
from sklearn.base import BaseEstimator, TransformerMixin
 
class RGB2GrayTransformer(BaseEstimator, TransformerMixin):
    """
    Convert an array of RGB images to grayscale
    """
 
    def __init__(self):
        pass
 
    def fit(self, X, y=None):
        """returns itself"""
        return self
 
    def transform(self, X, y=None):
        """perform the transformation and return an array"""
        return np.array([skimage.color.rgb2gray(img) for img in X])
class HogTransformer(BaseEstimator, TransformerMixin):
    """
    Expects an array of 2d arrays (1 channel images)
    Calculates hog features for each img
    """
 
    def __init__(self, y=None, orientations=9,
                 pixels_per_cell=(8, 8),
                 cells_per_block=(3, 3), block_norm='L2-Hys'):
        self.y = y
        self.orientations = orientations
        self.pixels_per_cell = pixels_per_cell
        self.cells_per_block = cells_per_block
        self.block_norm = block_norm
 
    def fit(self, X, y=None):
        return self
 
    def transform(self, X, y=None):
 
        def local_hog(X):
            return hog(X,
                       orientations=self.orientations,
                       pixels_per_cell=self.pixels_per_cell,
                       cells_per_block=self.cells_per_block,
                       block_norm=self.block_norm)
 
        try: # parallel
            return np.array([local_hog(img) for img in X])
        except:
            return np.array([local_hog(img) for img in X])

In [224]:
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import cross_val_predict
from sklearn.preprocessing import StandardScaler, Normalizer
import skimage
from skimage.feature import hog
# create an instance of each transformer

hogify = HogTransformer(pixels_per_cell=(27, 27), cells_per_block=(2,2), orientations=9, block_norm='L2-Hys'
)
scalify = StandardScaler()#  (27, 27)

X_train_hog = hogify.fit_transform(X_train)
X_train_prepared = scalify.fit_transform(X_train_hog)
print(X_train_prepared.shape)

(4800, 576)


In [225]:
from sklearn.svm import SVC
svm = SVC(C = 10.0)
svm.fit(X_train_prepared, y_train)

['ALPHA' 'BETA' 'CHI' 'DELTA' 'EPSILON' 'ETA' 'GAMMA' 'IOTA' 'KAPPA'
 'LAMDA' 'MU' 'NU' 'OMEGA' 'OMICRON' 'PHI' 'PI' 'PSI' 'RHO' 'SIGMA' 'TAU'
 'THETA' 'UPSILON' 'XI' 'ZETA']


In [230]:
X_test_hog = hogify.transform(223-X_test)   # use 255
X_test_prepared = scalify.transform(X_test_hog)
y_hat = svm.predict(X_test_prepared)
from sklearn.metrics import accuracy_score
print(accuracy_score(y_hat, y_test))

0.9658333333333333


In [53]:
#import pickle
#with open('svm_init_27_2_9.pkl', 'wb') as f:
#    svm = pickle.dump(svm, f)

In [231]:
from sklearn.metrics import classification_report
print(classification_report(y_hat, y_test))

              precision    recall  f1-score   support

       ALPHA       0.92      0.96      0.94        56
        BETA       0.97      0.96      0.96        67
         CHI       1.00      1.00      1.00        53
       DELTA       0.93      0.97      0.95        39
     EPSILON       1.00      1.00      1.00        65
         ETA       1.00      0.95      0.97        55
       GAMMA       0.96      0.93      0.95        59
        IOTA       0.98      0.93      0.95        54
       KAPPA       0.94      1.00      0.97        49
       LAMDA       0.96      0.94      0.95        47
          MU       0.97      0.97      0.97        37
          NU       1.00      0.93      0.97        45
       OMEGA       1.00      0.91      0.95        55
     OMICRON       0.93      0.95      0.94        40
         PHI       0.95      1.00      0.97        53
          PI       1.00      0.98      0.99        56
         PSI       0.94      1.00      0.97        34
         RHO       0.84    

In [112]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators = 500)
clf.fit(X_train_prepared, y_train)

RandomForestClassifier(n_estimators=500)

In [113]:
#X_test_hog = hogify.transform(X_test)
#X_test_prepared = scalify.transform(X_test_hog)
y_hat = clf.predict(X_test_prepared)
from sklearn.metrics import accuracy_score
print(accuracy_score(y_hat, y_test))

0.9575


In [131]:
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import decode_predictions
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.models import Model
from keras.layers import *
from keras.applications.inception_v3 import InceptionV3


input_ = Input((150, 150, 1))
x = Conv2D(32, 3, padding='same', activation='relu')(input_)
x = MaxPool2D()(x)
x = Conv2D(64, 3, activation='relu')(x)
x = Flatten()(x)

x = Dense(256,activation='relu')(x)
x = BatchNormalization()(x)
#x = Dropout(0.2)(x)
x = Dense(24, activation='softmax')(x)

model = Model(input_, x)
model.summary()

Model: "model_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_9 (InputLayer)        [(None, 150, 150, 1)]     0         
                                                                 
 conv2d_108 (Conv2D)         (None, 150, 150, 32)      320       
                                                                 
 max_pooling2d_11 (MaxPoolin  (None, 75, 75, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_109 (Conv2D)         (None, 73, 73, 64)        18496     
                                                                 
 flatten_7 (Flatten)         (None, 341056)            0         
                                                                 
 dense_15 (Dense)            (None, 256)               87310592  
                                                           

In [128]:
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import decode_predictions
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.models import Model
from keras.layers import *
from keras.applications.inception_v3 import InceptionV3

baseModel = InceptionV3(weights="imagenet", include_top=False,
	input_tensor=Input(shape=(150, 150, 3)))

headModel = baseModel.output
headModel = Flatten(name="flatten")(headModel)
#headModel = Dense(512, activation="relu")(headModel)
#headModel = Dropout(0.5)(headModel)
headModel = Dense(24, activation="softmax")(headModel)

model = Model(inputs=baseModel.input, outputs=headModel)
for layer in baseModel.layers[:-8]:
	layer.trainable = False
model.summary()



Model: "model_7"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_8 (InputLayer)           [(None, 150, 150, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_14 (Conv2D)             (None, 74, 74, 32)   864         ['input_8[0][0]']                
                                                                                                  
 batch_normalization_7 (BatchNo  (None, 74, 74, 32)  96          ['conv2d_14[0][0]']              
 rmalization)                                                                                     
                                                                                            

 batch_normalization_17 (BatchN  (None, 16, 16, 96)  288         ['conv2d_24[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 batch_normalization_18 (BatchN  (None, 16, 16, 32)  96          ['conv2d_25[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_5 (Activation)      (None, 16, 16, 64)   0           ['batch_normalization_12[0][0]'] 
                                                                                                  
 activation_7 (Activation)      (None, 16, 16, 64)   0           ['batch_normalization_14[0][0]'] 
                                                                                                  
 activatio

 conv2d_34 (Conv2D)             (None, 16, 16, 48)   13824       ['mixed1[0][0]']                 
                                                                                                  
 conv2d_37 (Conv2D)             (None, 16, 16, 96)   55296       ['activation_22[0][0]']          
                                                                                                  
 batch_normalization_27 (BatchN  (None, 16, 16, 48)  144         ['conv2d_34[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 batch_normalization_30 (BatchN  (None, 16, 16, 96)  288         ['conv2d_37[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activatio

                                                                                                  
 conv2d_48 (Conv2D)             (None, 7, 7, 128)    98304       ['mixed3[0][0]']                 
                                                                                                  
 batch_normalization_41 (BatchN  (None, 7, 7, 128)   384         ['conv2d_48[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_34 (Activation)     (None, 7, 7, 128)    0           ['batch_normalization_41[0][0]'] 
                                                                                                  
 conv2d_49 (Conv2D)             (None, 7, 7, 128)    114688      ['activation_34[0][0]']          
                                                                                                  
 batch_nor

                                                                                                  
 activation_44 (Activation)     (None, 7, 7, 160)    0           ['batch_normalization_51[0][0]'] 
                                                                                                  
 conv2d_59 (Conv2D)             (None, 7, 7, 160)    179200      ['activation_44[0][0]']          
                                                                                                  
 batch_normalization_52 (BatchN  (None, 7, 7, 160)   480         ['conv2d_59[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_45 (Activation)     (None, 7, 7, 160)    0           ['batch_normalization_52[0][0]'] 
                                                                                                  
 conv2d_55

 batch_normalization_62 (BatchN  (None, 7, 7, 160)   480         ['conv2d_69[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_55 (Activation)     (None, 7, 7, 160)    0           ['batch_normalization_62[0][0]'] 
                                                                                                  
 conv2d_65 (Conv2D)             (None, 7, 7, 160)    122880      ['mixed5[0][0]']                 
                                                                                                  
 conv2d_70 (Conv2D)             (None, 7, 7, 160)    179200      ['activation_55[0][0]']          
                                                                                                  
 batch_normalization_58 (BatchN  (None, 7, 7, 160)   480         ['conv2d_65[0][0]']              
 ormalizat

 conv2d_75 (Conv2D)             (None, 7, 7, 192)    147456      ['mixed6[0][0]']                 
                                                                                                  
 conv2d_80 (Conv2D)             (None, 7, 7, 192)    258048      ['activation_65[0][0]']          
                                                                                                  
 batch_normalization_68 (BatchN  (None, 7, 7, 192)   576         ['conv2d_75[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 batch_normalization_73 (BatchN  (None, 7, 7, 192)   576         ['conv2d_80[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activatio

 ormalization)                                                                                    
                                                                                                  
 batch_normalization_81 (BatchN  (None, 7, 7, 192)   576         ['conv2d_88[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_70 (Activation)     (None, 7, 7, 192)    0           ['batch_normalization_77[0][0]'] 
                                                                                                  
 activation_74 (Activation)     (None, 7, 7, 192)    0           ['batch_normalization_81[0][0]'] 
                                                                                                  
 conv2d_85 (Conv2D)             (None, 3, 3, 320)    552960      ['activation_70[0][0]']          
          

 activation_79 (Activation)     (None, 3, 3, 384)    0           ['batch_normalization_86[0][0]'] 
                                                                                                  
 activation_82 (Activation)     (None, 3, 3, 384)    0           ['batch_normalization_89[0][0]'] 
                                                                                                  
 activation_83 (Activation)     (None, 3, 3, 384)    0           ['batch_normalization_90[0][0]'] 
                                                                                                  
 batch_normalization_91 (BatchN  (None, 3, 3, 192)   576         ['conv2d_98[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_76 (Activation)     (None, 3, 3, 320)    0           ['batch_normalization_83[0][0]'] 
          

 batch_normalization_100 (Batch  (None, 3, 3, 192)   576         ['conv2d_107[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 activation_85 (Activation)     (None, 3, 3, 320)    0           ['batch_normalization_92[0][0]'] 
                                                                                                  
 mixed9_1 (Concatenate)         (None, 3, 3, 768)    0           ['activation_87[0][0]',          
                                                                  'activation_88[0][0]']          
                                                                                                  
 concatenate_1 (Concatenate)    (None, 3, 3, 768)    0           ['activation_91[0][0]',          
                                                                  'activation_92[0][0]']          
          

In [132]:
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=['accuracy'])

In [133]:
model.fit(x=X_train_pre, y=y_train_pre, batch_size=32, epochs=100, shuffle=True)

Epoch 1/100
300/300 [==============================] - 239s 795ms/step - loss: 0.6396 - accuracy: 0.3776
Epoch 2/100
300/300 [==============================] - 238s 791ms/step - loss: 0.5115 - accuracy: 0.4789
Epoch 3/100
300/300 [==============================] - 242s 805ms/step - loss: 0.6405 - accuracy: 0.4903
Epoch 4/100
300/300 [==============================] - 248s 827ms/step - loss: 0.8030 - accuracy: 0.4833
Epoch 5/100
300/300 [==============================] - 12114s 41s/step - loss: 0.9597 - accuracy: 0.4280
Epoch 6/100
 69/300 [=====>........................] - ETA: 2:58 - loss: 1.0508 - accuracy: 0.3347

KeyboardInterrupt: 

In [125]:
y_hat = model.predict(X_test_pre)

38/38 [==============================] - 4s 104ms/step


In [121]:
y_test.shape

(1200, 24)

In [122]:
from sklearn.metrics import accuracy_score
print(accuracy_score(le.inverse_transform(y_hat), le.inverse_transform(y_test)))
from sklearn.metrics import classification_report
print(classification_report(le.inverse_transform(y_hat), le.inverse_transform(y_test)))

0.27
              precision    recall  f1-score   support

       ALPHA       0.00      0.00      0.00         0
        BETA       0.03      1.00      0.06         2
         CHI       0.83      0.98      0.90        45
       DELTA       0.00      0.00      0.00         0
     EPSILON       0.00      0.00      0.00         0
         ETA       0.44      1.00      0.61        23
       GAMMA       0.18      1.00      0.30        10
        IOTA       0.00      0.00      0.00         0
       KAPPA       0.44      1.00      0.61        23
       LAMDA       0.80      0.93      0.86        40
          MU       0.00      0.00      0.00         0
          NU       0.02      1.00      0.05         1
       OMEGA       0.00      0.00      0.00         0
     OMICRON       0.00      0.00      0.00         0
         PHI       0.34      1.00      0.51        19
          PI       0.40      1.00      0.57        22
         PSI       0.00      0.00      0.00         0
         RHO       0.7

C:\Users\ambat\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ambat\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ambat\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, le

In [37]:
model.save_weights("initial_rot_256_try_2_thresh_35.h5")

In [ ]:
import os, sys
import numpy as np
import cv2
import time
from imutils.object_detection import non_max_suppression

def east_detect(image):
    layerNames = [
    	"feature_fusion/Conv_7/Sigmoid",
    	"feature_fusion/concat_3"]
    
    orig = image.copy()
    
    if len(image.shape) == 2:
        image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
    
    (H, W) = image.shape[:2]
    
    # set the new width and height and then determine the ratio in change
    # for both the width and height: Should be multiple of 32
    (newW, newH) = (320, 320)
    
    rW = W / float(newW)
    rH = H / float(newH)
    
    # resize the image and grab the new image dimensions
    image = cv2.resize(image, (newW, newH))
    
    (H, W) = image.shape[:2]
    
    net = cv2.dnn.readNet("frozen_east_text_detection.pb")
    
    blob = cv2.dnn.blobFromImage(image, 1.0, (W, H),
    	(123.68, 116.78, 103.94), swapRB=True, crop=False)
    
    start = time.time()
    
    net.setInput(blob)
    
    (scores, geometry) = net.forward(layerNames)
    
    (numRows, numCols) = scores.shape[2:4]
    rects = []
    confidences = []
    # loop over the number of rows
    for y in range(0, numRows):
        # extract the scores (probabilities), followed by the geometrical
        # data used to derive potential bounding box coordinates that
        # surround text
        scoresData = scores[0, 0, y]
        xData0 = geometry[0, 0, y]
        xData1 = geometry[0, 1, y]
        xData2 = geometry[0, 2, y]
        xData3 = geometry[0, 3, y]
        anglesData = geometry[0, 4, y]
    
        for x in range(0, numCols):
    		# if our score does not have sufficient probability, ignore it
            # Set minimum confidence as required
            if scoresData[x] < 0.5:
                continue
    		# compute the offset factor as our resulting feature maps will
            #  x smaller than the input image
            (offsetX, offsetY) = (x * 4.0, y * 4.0)
            # extract the rotation angle for the prediction and then
            # compute the sin and cosine
            angle = anglesData[x]
            cos = np.cos(angle)
            sin = np.sin(angle)
            # use the geometry volume to derive the width and height of
            # the bounding box
            h = xData0[x] + xData2[x]
            w = xData1[x] + xData3[x]
            # compute both the starting and ending (x, y)-coordinates for
            # the text prediction bounding box
            endX = int(offsetX + (cos * xData1[x]) + (sin * xData2[x]))
            endY = int(offsetY - (sin * xData1[x]) + (cos * xData2[x]))
            startX = int(endX - w)
            startY = int(endY - h)
            # add the bounding box coordinates and probability score to
            # our respective lists
            rects.append((startX, startY, endX, endY))
            confidences.append(scoresData[x])
                        
    boxes = non_max_suppression(np.array(rects), probs=confidences)
    # loop over the bounding boxes
    for (startX, startY, endX, endY) in boxes:
    	# scale the bounding box coordinates based on the respective
    	# ratios
    	startX = int(startX * rW)
    	startY = int(startY * rH)
    	endX = int(endX * rW)
    	endY = int(endY * rH)
    	# draw the bounding box on the image
    	cv2.rectangle(orig, (startX, startY), (endX, endY), (0, 255, 0), 2)
    
    
    print(time.time() - start)
    return orig

image = cv2.imread("0.png")

out_image = east_detect(image)

cv2.imwrite("sample_output.jpg", out_image)

In [ ]:
RETR_CCOMP